In [3]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from time import sleep
from selenium.webdriver.common.by import By
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
url = 'https://phongvu.vn/c/laptop'
driver.get(url)
wait = WebDriverWait(driver, 30)

def is_view_more_button_displayed():
    try:
        view_more_button = wait.until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.css-1wwh4ob .button-text'))
        )
        return view_more_button.is_displayed()
    except:
        return False

# Vòng lặp để click vào nút "Xem thêm sản phẩm" cho đến khi nó không còn hiển thị
while is_view_more_button_displayed():
    view_more_button = wait.until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '.css-1wwh4ob .button-text'))
    )
    try:
        driver.execute_script("arguments[0].click();", view_more_button)
    except ElementClickInterceptedException:
        # Xử lý pop-up hoặc các phần tử chặn khác
        try:
            close_popup_button = WebDriverWait(driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, '.css-73p2ms'))
            )
            close_popup_button.click()
            sleep(2)  # Chờ một khoảng thời gian sau khi đóng pop-up
        except:
            pass
        # Di chuyển đến phần tử lại
        driver.execute_script("arguments[0].scrollIntoView();", view_more_button)
        # Thử click bằng JavaScript một lần nữa
        driver.execute_script("arguments[0].click();", view_more_button)

    # Chờ một khoảng thời gian sau khi click
    sleep(3)

product_links = WebDriverWait(driver, 10).until(EC.visibility_of_all_elements_located((By.CLASS_NAME, 'product-card')))
# Trích xuất thuộc tính href từ các liên kết
all_product_links = [link.find_element(By.TAG_NAME, 'a').get_attribute('href') for link in product_links]

print(all_product_links)

driver.quit()

['https://phongvu.vn/laptop-acer-aspire-3-a315-59-381e-nx-k6tsv-006--s220909328', 'https://phongvu.vn/lenovo-ideapad-3-15iau7-82rk001qvn--s220700824', 'https://phongvu.vn/may-tinh-xach-tay-laptop-acer-aspire-3-a314-23m-r4tx-nx-kexsv-001-amd-ryzen-5-7520u-bac--s230402849', 'https://phongvu.vn/may-tinh-xach-tay-laptop-acer-aspire-3-a315-59-51x8-nx-k6tsv-00f-i5-1235u-bac--s221003814', 'https://phongvu.vn/laptop-asus-vivobook-go-15-e1504fa-nj454w-amd-ryzen-5-7520u--s230500521', 'https://phongvu.vn/lenovo-ideapad-5-15ial7-82sf006lvn--s220801877', 'https://phongvu.vn/acer-aspire-7-a715-42g-r05g-nh-qaysv-007--s211104709', 'https://phongvu.vn/laptop-lenovo-ideapad-5-14iah8-83bf002nvn-i5-12450h-xam--s230704076', 'https://phongvu.vn/may-tinh-xach-tay-laptop-lenovo-ideapad-slim-5-light-14abr8-amd-ryzen-7-7730u-xam--s230702516', 'https://phongvu.vn/may-tinh-xach-tay-laptop-acer-aspire-7-a715-76-53pj-nh-qgesv-007-i5-12450h-den--s230904124', 'https://phongvu.vn/may-tinh-xach-tay-laptop-acer-aspire-5

In [4]:
print(len(all_product_links))

753


In [ ]:
import re
import requests
from time import sleep
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

def get_product_info(all_product_links):
    driver.get(all_product_links)
    driver.implicitly_wait(10)  # Chờ tối đa 10 giây

# Cuộn xuống cuối trang
    driver.find_element_by_tag_name('body').send_keys(Keys.END)
    time.sleep(5)
    html_content = driver.page_source
    soup = BeautifulSoup(html_content, 'html.parser')
    try:
        brand_element = soup.find('div', class_ = 'teko-col teko-col-4 css-17ajfcv').find('div', {'type': 'body', 'class': 'css-1lchwqw'}, string='Thương hiệu')
        if brand_element.find_next_sibling('div').text.strip() == 'APPLE':
            part_number_element = soup.find('div', class_ = 'teko-col teko-col-4 css-17ajfcv').find('div', {'type': 'body', 'class': 'css-1lchwqw'}, string='Tên')
            if part_number_element:
                text = part_number_element.find_next_sibling().text.strip()
                id = text.split()
                product_id = id[-1].strip()
        else:
            part_number_element = soup.find('div', class_ = 'teko-col teko-col-4 css-17ajfcv').find('div', {'type': 'body', 'class': 'css-1lchwqw'}, string='Part-number')

            # Kiểm tra xem có phần tử "Part-number" được tìm thấy không
            if part_number_element:
                product_id = part_number_element.find_next_sibling('div').text.strip()
            else:
                product_id = ''
    except:
        product_id = ''
    
    #TÊN SẢN PHẨM
    try:
        product_name = soup.find('h1', class_='css-nlaxuc').text.strip().replace('Liên hệ đặt hàng','')
    except:
        product_name = ''
    
    #URL SẢN PHẨM
    try:
        product_url = soup.find('link', {'rel': 'canonical'}).get('href')
    except:
        product_url = ''
    
    #GIÁ KHUYẾN MẠI
    try:
        discounted_price = soup.find('div', class_ = 'css-2zf5gn').find('div', class_ = 'att-product-detail-latest-price css-roachw').text.strip()
    except:
        discounted_price = ''
    
    #GIÁ GỐC
    try:
        original_price = soup.find('div', class_ = 'css-2zf5gn').find('div', class_ = 'att-product-detail-retail-price css-18z00w6').text.strip()
    except:
        original_price = discounted_price
    
    #PHẦN TRĂM GIẢM GIÁ
    try:   
        discount_percentage = soup.find('div', class_ = 'css-2zf5gn').find('div', class_ = 'css-2rwx6s').text.strip().replace('-','')
    except:
        discount_percentage = ''
    
    #KHUYẾN MẠI ĐI KÈM
    promotion_values =  soup.find_all(class_='css-1lfpde8')
    
        # Tạo một từ điển để lưu các giá trị
    promotions = []
    try:
        for i, promotion in enumerate(promotion_values, start=1):
        # Trích xuất thông tin từ mỗi phần tử và thêm vào từ điển
            variable_name = f"Promotion__{i}"
            promotion_text = promotion.text.strip()
            promotions.append((variable_name,promotion_text ))
    except AttributeError:
        pass
    
    try:
        promotion_more_value = soup.find('div', class_ = 'BOXKHUYENMAILIENQUAN').find_all('li')
        for i, promotion_more in enumerate(promotion_more_value, start=1):
        # Trích xuất thông tin từ mỗi phần tử và thêm vào từ điển
            promotion_more_name = f"Promotion__{i}"
            promotion_more_text = promotion_more.text.strip().replace('Xem chi tiết','')
            promotions.append((promotion_more_name,promotion_more_text ))
    except AttributeError:
        pass   
    
    #TRẢ GÓP
    new_response = requests.get('https://phongvu.vn/cong-nghe/cach-mua-hang-tra-gop-tai-phong-vu')
    new_soup = BeautifulSoup(new_response.text, 'html.parser')
    match = re.search(r'Phí trả góp:\s*(.*?)</p>', str(new_soup))
    installment_payment = match.group(1).strip().replace('</strong>', '')
    max_promotion = 12

    #TẠO DICTIONARY LƯU TRỮ
    product_price = {
            'product_id': product_id,
            'product_name': product_name,
            'discounted_price': discounted_price,
            'original_price': original_price,
            'discount_percentage': discount_percentage,
            'installment_payment' : installment_payment,
            'shop_brand': 'Phong Vũ'
        }
    # Thêm các biến khuyến mại vào dictionary
    for i in range(1, max_promotion + 1):
        key = f'promotion_{i}'
        # Nếu i vượt quá số lượng thực sự của khuyến mại, thiết lập giá trị là None
        if i <= len(promotions):
                value = promotions[i-1][1]
        else:
            value = ''

        product_price[key] = value

    product_price['product_url'] = product_url

    return product_price

driver = webdriver.Chrome(executable_path="C:\\Users\\TECHCARE\\Desktop\\crawl_project\\chromedriver-win64\\chromedriver.exe")
# Duyệt qua các đường link sản phẩm và lấy thông tin
data = []
for product_link in all_product_links:
    product_price = get_product_info(product_link)
    data.append(product_price)
    print(product_price)

driver.quit()

{'product_id': 'NX.K6TSV.006', 'product_name': 'Laptop Acer Aspire 3 A315-59-381E (NX.K6TSV.006) (i3-1215U) (Bạc)', 'discounted_price': '10.090.000₫', 'original_price': '11.990.000₫', 'discount_percentage': '15.847%', 'installment_payment': '0.99%/ tháng', 'shop_brand': 'Phong Vũ', 'promotion_1': '1x Túi đeo lưng/ Balo laptop Targus 15.6 TSB883 Black (Safire) (Quà tặng - logo Phong Vũ)', 'promotion_2': '1x Mã giảm thêm 50.000đ cho một số ổ cứng SSD và HDD WD 1TB', 'promotion_3': '1x Mã giảm thêm 600.000đ cho màn hình LG Gram View', 'promotion_4': '1x Mã giảm thêm 150.000đ cho một số chuột Logitech, Asus, Newmen, tai nghe Zidli', 'promotion_5': '1x Mã giảm thêm 100.000 mua Phần mềm Microsoft M365 Personal (QQ2-00003), M365 Family (6GQ-00083)', 'promotion_6': '1x Mã giảm thêm 150.000 mua Phần mềm Microsoft Office Home and Student 2021 (79G-05387)', 'promotion_7': 'Nhập mã PVACER240301 giảm 300.000đ  cho đơn hàng  có sản phẩm này', 'promotion_8': 'Giảm thêm đến  200.000đ   dành cho Học si

In [46]:
import pandas as pd
df = pd.DataFrame(data)
df

,product_id,product_name,discounted_price,original_price,discount_percentage,installment_payment,shop_brand,promotion_1,promotion_2,promotion_3,promotion_4,promotion_5,promotion_6,promotion_7,promotion_8,promotion_9,promotion_10,promotion_11,promotion_12,product_url
0,NX.K6TSV.006,Laptop Acer Aspire 3 A315-59-381E (NX.K6TSV.00...,10.090.000₫,11.990.000₫,15.847%,0.99%/ tháng,Phong Vũ,1x Túi đeo lưng/ Balo laptop Targus 15.6 TSB88...,1x Mã giảm thêm 50.000đ cho một số ổ cứng SSD ...,1x Mã giảm thêm 600.000đ cho màn hình LG Gram ...,1x Mã giảm thêm 150.000đ cho một số chuột Logi...,1x Mã giảm thêm 100.000 mua Phần mềm Microsoft...,1x Mã giảm thêm 150.000 mua Phần mềm Microsoft...,Nhập mã PVACER240301 giảm 300.000đ cho đơn hà...,Giảm thêm đến 200.000đ dành cho Học sinh - ...,"Ưu đãi trả góp: Lãi suất chỉ 0.99%, trả trước ...",Nhập mã QRPV2024 - Giảm 100.000đ cho đơn từ...,Nhập mã ZLPPV : 1. Giảm thêm 50.000đ cho đơ...,,https://phongvu.vn/laptop-acer-aspire-3-a315-5...
1,82RK001QVN,Laptop Lenovo IdeaPad 3 15IAU7 - 82RK001QVN (i...,16.790.000₫,17.990.000₫,6.67%,0.99%/ tháng,Phong Vũ,1x Mã giảm thêm 50.000đ cho một số ổ cứng SSD ...,1x Mã giảm thêm 600.000đ cho màn hình LG Gram ...,1x Mã giảm thêm 150.000đ cho một số chuột Logi...,1x Mã giảm thêm 100.000 mua Phần mềm Microsoft...,1x Mã giảm thêm 150.000 mua Phần mềm Microsoft...,Giảm thêm đến 200.000đ dành cho Học sinh - ...,"Ưu đãi trả góp: Lãi suất chỉ 0.99%, trả trước ...",Nhập mã QRPV2024 - Giảm 100.000đ cho đơn từ...,Nhập mã ZLPPV : 1. Giảm thêm 50.000đ cho đơ...,,,,https://phongvu.vn/lenovo-ideapad-3-15iau7-82r...
2,NX.KEXSV.001,Laptop Acer Aspire 3 A314-23M-R4TX (NX.KEXSV.0...,11.990.000₫,13.290.000₫,9.782%,0.99%/ tháng,Phong Vũ,1x Túi đeo lưng/ Balo laptop Targus 15.6 TSB88...,1x Mã giảm thêm 50.000đ cho một số ổ cứng SSD ...,1x Mã giảm thêm 600.000đ cho màn hình LG Gram ...,1x Mã giảm thêm 150.000đ cho một số chuột Logi...,1x Mã giảm thêm 100.000 mua Phần mềm Microsoft...,1x Mã giảm thêm 150.000 mua Phần mềm Microsoft...,Nhập mã PVACER240301 giảm 300.000đ cho đơn hà...,Giảm thêm đến 200.000đ dành cho Học sinh - ...,"Ưu đãi trả góp: Lãi suất chỉ 0.99%, trả trước ...",Nhập mã QRPV2024 - Giảm 100.000đ cho đơn từ...,Nhập mã ZLPPV : 1. Giảm thêm 50.000đ cho đơ...,,https://phongvu.vn/may-tinh-xach-tay-laptop-ac...
3,NX.K6TSV.00F,Laptop Acer Aspire 3 A315-59-51X8 (NX.K6TSV.00...,12.990.000₫,15.490.000₫,16.139%,0.99%/ tháng,Phong Vũ,1x Túi đeo lưng/ Balo laptop Targus 15.6 TSB88...,1x Mã giảm thêm 50.000đ cho một số ổ cứng SSD ...,1x Mã giảm thêm 600.000đ cho màn hình LG Gram ...,1x Mã giảm thêm 150.000đ cho một số chuột Logi...,1x Mã giảm thêm 100.000 mua Phần mềm Microsoft...,1x Mã giảm thêm 150.000 mua Phần mềm Microsoft...,Nhập mã PVACER240301 giảm 300.000đ cho đơn hà...,Giảm thêm đến 200.000đ dành cho Học sinh - ...,"Ưu đãi trả góp: Lãi suất chỉ 0.99%, trả trước ...",Nhập mã QRPV2024 - Giảm 100.000đ cho đơn từ...,Nhập mã ZLPPV : 1. Giảm thêm 50.000đ cho đơ...,,https://phongvu.vn/may-tinh-xach-tay-laptop-ac...
4,90NB0ZR1-M00NB0,Laptop Asus Vivobook Go 15 E1504FA-NJ454W (AMD...,12.490.000₫,14.490.000₫,13.803%,0.99%/ tháng,Phong Vũ,1x Túi đeo lưng/ Balo laptop Targus 15.6 TSB88...,1x Chuột vi tính Logitech M171 Wireless/Trắng ...,1x Mã giảm thêm 50.000đ cho một số ổ cứng SSD ...,1x Mã giảm thêm 600.000đ cho màn hình LG Gram ...,1x Mã giảm thêm 150.000đ cho một số chuột Logi...,1x Mã giảm thêm 100.000 mua Phần mềm Microsoft...,1x Mã giảm thêm 150.000 mua Phần mềm Microsoft...,Giảm thêm đến 200.000đ dành cho Học sinh - ...,"Ưu đãi trả góp: Lãi suất chỉ 0.99%, trả trước ...",Nhập mã QRPV2024 - Giảm 100.000đ cho đơn từ...,Nhập mã ZLPPV : 1. Giảm thêm 50.000đ cho đơ...,,https://phongvu.vn/laptop-asus-vivobook-go-15-...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
748,621VN,Laptop MSI Cyborg 15 A12UC - 621VN (i5-12450H/...,18.990.000₫,19.990.000₫,5.003%,0.99%/ tháng,Phong Vũ,1x Mã giảm thêm 50.000đ cho một số ổ cứng SSD ...,1x Mã giảm thêm 600.000đ

In [47]:
df.to_excel('phongvu_price.xlsx', index=False)